In [18]:
import requests
from bs4 import BeautifulSoup
url ='https://flightaware.com/live/findflight?origin=KSEA&destination=KLAX'
response = requests.get(url)
page = response.text

#soup = BeautifulSoup(page,"lxml")

In [19]:
response.status_code

200

In [50]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

import os
import dateutil.parser
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
chromedriver = "/users/Minmin/Downloads/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

#driver = webdriver.Chrome(chromedriver)

#driver.get(url)

In [51]:
options= Options()
#options.add_argument("--headless") # Runs Chrome in headless mode.
options.add_argument('--allow-no-sandbox-job') # Bypass OS security model
options.add_argument("--test-type")
options.add_argument('--disable-gpu')  # applicable to windows os only
options.add_argument('start-maximized') # 
options.add_argument('disable-infobars')
#options.add_argument("--disable-extensions")
options.add_argument("--disable-web-security")
options.add_argument("--allow-running-insecure-content")
options.add_argument("--safebrowsing-disable-extension-blacklist")
options.add_argument("--safebrowsing-disable-download-protection")
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument('--disable-bundled-ppapi-flash')

options.add_extension(("/users/Minmin/Downloads/Adblock-Plus_v3.1.crx"))

In [52]:
driver = webdriver.Chrome(chromedriver,chrome_options=options)

In [53]:
driver.get(url)

In [54]:
#log in
driver.find_element_by_xpath('//*[@id="topWrapper"]/div[1]/nav/div/div[2]/a').click()
driver.find_element_by_xpath('//*[@id="popupLogin"]/div/div[2]/div/form/div[1]/input[1]').send_keys('jmhan0428')
driver.find_element_by_xpath('//*[@id="popupLogin"]/div/div[2]/div/form/div[1]/input[2]').send_keys('123456')
driver.find_element_by_xpath('//*[@id="popupLogin"]/div/div[2]/div/form/button').click()

In [27]:
destination_list=['KLAX','KSFO','PANC','KDEN','KLAS','KPHX','KORD','KPDX','KDFW','KSJC']

In [29]:
# define function to generate flightid
def get_flight(x):
    driver.get(x)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    len_flight=len(soup.find_all('table')[1].find_all('tr'))
    row_num=list(range(1,len_flight,2))
    la_flight=[]
    for i in row_num:
        fl=soup.find_all('table')[1].find_all('tr')[i].find_all('td')[1].text.strip()
        la_flight.append(fl)
    la_flight_uni = sorted(list(set(la_flight)))

    return la_flight_uni

In [30]:
#get all unique flightid for destination airports above
city_url_list=[]
for i in destination_list:
    temp_list = get_flight("https://flightaware.com/live/findflight?origin=KSEA&destination="+i)
    city_url_list.append(temp_list)
    time.sleep(8)

In [36]:
#combine all the flightid into one list
import itertools
flightid_list=list(itertools.chain.from_iterable(city_url_list))


['ABX1817',
 'ASA1780',
 'ASA1782',
 'ASA1784',
 'ASA1786',
 'ASA1788',
 'ASA1790',
 'ASA1792',
 'ASA1794',
 'ASA1796',
 'ASA1798',
 'ASA286',
 'ASA315',
 'ASA370',
 'ASA394',
 'ASA404',
 'ASA416',
 'ASA432',
 'ASA446',
 'ASA466',
 'ASA9246',
 'CPZ3269',
 'CPZ6048',
 'CPZ6049',
 'CPZ6052',
 'CPZ6054',
 'CPZ6058',
 'DAL1228',
 'DAL128',
 'DAL1437',
 'DAL1587',
 'DAL2077',
 'DAL2101',
 'DAL2135',
 'DAL2596',
 'DAL2722',
 'DAL2988',
 'DAL379',
 'DAL882',
 'DAL983',
 'FDX3034',
 'GEC8230',
 'NKS726',
 'SKW5191',
 'SKW5384',
 'ASA1740',
 'ASA1742',
 'ASA1744',
 'ASA1746',
 'ASA1748',
 'ASA1750',
 'ASA1752',
 'ASA1754',
 'ASA1756',
 'ASA1758',
 'ASA1762',
 'ASA300',
 'ASA376',
 'ASA382',
 'ASA9347',
 'ASA9802',
 'DAL1092',
 'DAL1806',
 'DAL2010',
 'DAL2109',
 'DAL2322',
 'DAL2577',
 'DAL2684',
 'DAL2698',
 'DAL2770',
 'DAL2983',
 'DAL399',
 'DAL659',
 'UAL1560',
 'UAL213',
 'UAL214',
 'UAL2356',
 'UAL294',
 'UAL587',
 'UAL659',
 'UAL698',
 'UAL816',
 'ASA101',
 'ASA105',
 'ASA107',
 'ASA111'

In [37]:
#pickle master flightid list
import pickle
with open('master_flightid_list.pickle', 'wb') as f:
    pickle.dump(flightid_list, f, pickle.HIGHEST_PROTOCOL)

### The following steps are to generate a list of all flights during the past three months

In [ ]:
driver.get('https://flightaware.com/live/flight/ABX1817/history/160')

In [55]:
#generate a master list of all url for each flight each day
url_all_flight=[]
for i in flightid_list:
    driver.get('https://flightaware.com/live/flight/'+i+'/history/160')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table=soup.find(class_='prettyTable fullWidth tablesaw tablesaw-stack')
    rows=[row for row in table.find_all('a')]
    url_part=rows[0:len(rows):3]
    
    url_per_flight=[]
    for j in url_part:
        if 'Z/KSEA'in str(j):
            j= str(j).split('>')[0].replace("<a href=",'').replace('"','')
            url_per_flight.append('https://flightaware.com'+j)
    time.sleep(8)
    url_all_flight.append(url_per_flight)

In [56]:
url_all_flight=list(itertools.chain.from_iterable(url_all_flight))

In [58]:
# pickle master list url
with open('master_url_list.pickle', 'wb') as f:
    pickle.dump(url_all_flight, f, pickle.HIGHEST_PROTOCOL)

### The following steps are to scrape info from each web page

In [71]:
def get_info_page(url):

    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    data={}
    #find airline
    airline=soup.find('title').text.split()[0]
    #find flightid
    flightid=soup.find('title').text.split()[2].replace('(','').replace(')','')+soup.find('title').text.split()[3].replace('#','')
    #find flight date
    date=dateutil.parser.parse(soup.find('title').text.split()[5])
    #find destination airport
    destination=soup.find('title').text.split()[9]
    #find departure gate
    #gate=soup.find_all('strong')[3].text.split()[1]
    #find scheduled  Departure time
    scheduled_departure=soup.find_all(text="Scheduled ")[0].findNextSibling().text.strip()
    #find actual departure time
    actual_departure=soup.find_all(class_='flightPageDataActualTimeText')[0].text.strip()
    #find actual takeoff time
    actual_takeoff=soup.find_all(class_='flightPageDataActualTimeText')[1].text.strip()
    #find scheduled  Departure time
    scheduled_takeoff=soup.find_all(text="Scheduled ")[0].findNextSibling().text.strip()
    #find aircraft type
    aircraft=soup.find(class_='flightPageData').text.strip().split('\xa0')[0]
    #find taxi time
    #taxi_time=soup.find(text="Taxi Time: ").findNextSibling().text.strip()
    #find delay status
    delay=soup.find(class_='flightPageOriginDelayStatus').text.strip().replace('(','').replace(')','')
    #find distance
    distance =soup.find_all(class_='flightPageData')[9].text.split(':')[2].split(' ')[1]
    data[date]=[date,airline,aircraft,flightid,destination,distance,scheduled_departure,actual_departure,scheduled_takeoff,actual_takeoff,delay]
    df=pd.DataFrame(data).T
    return df

In [73]:
all_flight=[]
none=pd.DataFrame()
for i in url_all_flight[1:]:
    try:
        all_flight.append(get_info_page(i))
    except:
        all_flight.append(none)
    time.sleep(8)

In [74]:
result = pd.concat(all_flight)

In [75]:
result

,0,1,2,3,4,5,6,7,8,9,10
2018-07-12,2018-07-12 00:00:00,ABX,Boeing 767-200 (twin-jet),GB1817,KLAX,"1,041",08:34PM PDT,08:43PM PDT,08:34PM PDT,11:18PM PDT,on time
2018-07-11,2018-07-11 00:00:00,ABX,Boeing 767-300 (twin-jet),GB1817,KLAX,"1,061",08:34PM PDT,09:23PM PDT,08:34PM PDT,11:30PM PDT,49 minutes late
2018-07-10,2018-07-10 00:00:00,ABX,Boeing 767-200 (twin-jet),GB1817,KSEA,482,08:34PM PDT,09:10PM PDT,08:34PM PDT,10:37PM PDT,36 minutes late
2018-07-10,2018-07-10 00:00:00,ABX,Boeing 767-200 (twin-jet),GB1817,KLAX,482,08:34PM PDT,09:10PM PDT,08:34PM PDT,10:37PM PDT,36 minutes late
